# Homework 1 - Streaming and Higher Order Functions (15 pts)

Please turn in your note book naming as *BDM\_HW1\_LastName.ipynb*.


## Task 1 (2 points)

Please complete the **lambda f1** definition below by filling in the _________ part. **f1** is expected to take a single string argument and returns whether the string can be converted to a natural number or zero. If it's possible, returns **'Number'**, otherwise returns **'Not a number'**. For example *'0123'* is a valid number 'Number', whereas *'0xff'* is not.

In [ ]:
f1 = lambda x: 'Number' if x.isdigit() else 'Not a number'
print(f1('1A'))
print(f1('12'))
print(f1('b1'))

Not a number
Number
Not a number



## Task 2 (2 points)

Please complete the **lambda f2** definition below by filling in the _________ part. **f2** takes a single iterable (e.g. a list or a string), and returns the number of unique elements. Please see the sample output below.

In [ ]:
f2 = lambda x: len(set(x))
print(f2([1,2,3,4,1,5,2]))
print(f2('hello world'))

5
8



## Task 3 (2 points)

Please complete the **lambda f3** definition below by filling in the _________ part. **f3** takes two strings *x* and *y*, and return all the words in *x* that do not appear in *y*. Please note that the word comparison iscase insensitive. Samples area also provided below.

In [ ]:
f3 = lambda x, y: [i for i in x.split(' ') if i not in y.split(' ')]
print(f3('big data management and analysis', 'big data computing'))
print(f3('this is a phrase', 'this is another phrase'))

['management', 'and', 'analysis']
['a']



## Task 4 (9 points)

We are greatly inspired by the [Consumer Complaints](https://github.com/InsightDataScience/consumer_complaints) challenge from [InsightDataScience](https://github.com/InsightDataScience/). In fact, we are going to tackle the same challenge but streaming. Please read through the challenge at the following link (the most important sections are “Input dataset” and “Expected output”):

<https://github.com/InsightDataScience/consumer_complaints>

Our problem is different than the one above in that the INPUT is also <ins>sorted by product (alphabetically) and year (ascending)</ins>, similar to the requested output format. You are asked to complete the generator below to produce the expected output. You should not edit any other code.

### INPUT
Your code will be evaluated against the original data set (in CSV format) downloaded from the below link (of course with a sorted version). For your convenience, a smaller version of the file is provided below for testing purposes.

https://www.consumerfinance.gov/data-research/consumer-complaints/#download-the-data



In [ ]:
!gdown --id 1-IeoZDwT5wQzBUpsaS5B6vTaP-2ZBkam -O complaints.csv
!head -n 1 complaints.csv

Downloading...
From: https://drive.google.com/uc?id=1-IeoZDwT5wQzBUpsaS5B6vTaP-2ZBkam
To: /content/complaints.csv
100% 3.91M/3.91M [00:00<00:00, 34.1MB/s]
Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID


In [ ]:
from functools import reduce
from functools import cmp_to_key

def modified_year(x):
  x['Date received'] = x['Date received'].split('-')[0]
  return x;

def add_count(x):
  x['Count'] = 1
  return x

def merge_record(x, y):
  for rec in x:
    if rec['Product'] == y['Product'] and rec['Date received'] == y['Date received'] and rec['Company'] == y['Company']:
      rec['Count'] += 1
      return x
  
  x.append(y)
  return x

def count_company(rec):
  num = reduce(lambda x: x+1, filter(lambda x: x['Product'] == rec['Product'] and x['year'] == rec['year'], data4), 0)
  rec['Count company'] = num
  return rec


def delete_key(x):
  del x['Company']
  del x['Count']
  del x['Percentage']
  return x


def merge_result(x, y):
  for i in x:
    if i['Product'] == y['Product'] and i['Date received'] == y['Date received']:
      return x;
  x.append(y)
  return x


def task4(reader):
  # YOU CAN ONLY EDIT CODE BELOW THIS LINE
  # Note: below is just an example, and not correct
  
  # modified the date
  reader = map(modified_year, reader)
  reader = map(add_count, reader)
  reader = reduce(merge_record, reader, [])
  
  # count # of company to be complained for each product and year
  # count total cases for each product and year
  # calculate percentage for each comapny to be complained in that product and year
  for row in reader:
    row['Count company'] = len(list(filter(lambda x: x['Product'] == row['Product'] and x['Date received'] == row['Date received'], reader)))
    row['Total count'] = reduce(lambda x,y: x+y, map(lambda x: x['Count'], filter(lambda x: x['Product'] == row['Product'] and x['Date received'] == row['Date received'], reader)), 0)
    row['Percentage'] = int(round(row['Count'] / row['Total count'], 2)*100)

  # find the highest percentage of company correspond to product and year for each row
  for row in reader:
    row['Highest percentage'] = max(list(map(lambda x: x['Percentage'], filter(lambda x: x['Product'] == row['Product'] and x['Date received'] == row['Date received'], reader))))

  reader = reduce(merge_result, map(delete_key, reader), [])

  for row in reader: # this line should stay the same
    yield [row['Product'].lower(), row['Date received'], row['Total count'], row['Count company'], row['Highest percentage']]



In [ ]:
def task4(reader):
  next_row=next(reader)
  product=next_row['Product'].lower()
  year = next_row['Date received'].split('-')[0]
  complaint=1
  company={next_row['Company']:1}

  for row in reader:
    if year == row['Date received'].split('-')[0] and next_row['Company'] != row['Company']:
      complaint += 1
      company[row['Company']]=company.get(row['Company'],0)+1
    else:
      yield [product, year, complaint, max(company.values()), round(max(company.values())/complaint*100)]
      product=row['Product'].lower()
      year = row['Date received'].split('-')[0]
      complaint=1
      company[row['Company']]=company.get(row['Company'],0)+1


# DO NOT EDIT THIS CELL, YOU CAN RUN TO QUICKLY CHECK YOUR RESULTS

import csv

with open('complaints.csv', 'r') as fi, open('output.csv', 'w') as fo:
  reader = csv.DictReader(fi)
  writer = csv.writer(fo)
  for row in task4(reader):
    writer.writerow(row)

!head output.csv

<generator object task4 at 0x7fb1338d58d0>

In [ ]:
# DO NOT EDIT THIS CELL, YOU CAN RUN TO QUICKLY CHECK YOUR RESULTS

import csv

with open('complaints.csv', 'r') as fi, open('output.csv', 'w') as fo:
  reader = csv.DictReader(fi)
  writer = csv.writer(fo)
  task4(reader)
  #for row in task4(reader):
    #writer.writerow(row)

#!head output.csv